In [ ]:
#run to install ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#run to pull mistral model locally/to download any model locally
ollama run mistral:instruct

In [ ]:
llm = "mistral:instruct"